In [795]:
first_frame_blower = 4520 # first frame that the blower leaves
first_frame_coyote = 4615 # first frame that the coyote leaves
end_first_acceleration_blower = 4695 # at this point, both the blower and the coyote move at the same speed
end_first_acceleration_coyote = 4689

bursts = (4740, 4750, 4765, 4784, 4805, 4845, 4868, 4879, 4906, 4918, 4929, 4936, 4941, 4948, 4952, 4958) # at these points, the blower shoots out a ball to accelerate further

wall_hit_frame = 4983 # last frame just before blower hits wall
catchup_frame = 5268 # at this point, coyote stops and seems to be pretty close to the blower

In [796]:
height_body = 1.295 # height of blower in meters
radius_body = 0.94 / 2 # radius of blower in meters

height_top = 0.147 # height of top of blower in meters
radius_top = 0.474 / 2 # radius of top of blower in meters

distance_between_two = 6.83 # distance between the two when they move at the same speed

In [797]:
import math

total_time_blower = (wall_hit_frame - first_frame_blower) / 30 # total time in seconds
total_time_coyote = (catchup_frame - first_frame_coyote) / 30

max_speed_coyote = 43 / 2.237 # max speed of animal converted from mph to m/s

first_acceleration_blower = max_speed_coyote / ((end_first_acceleration_blower - first_frame_blower) / 30) # acceleration in m/s^2
first_acceleration_coyote = max_speed_coyote / ((end_first_acceleration_coyote - first_frame_coyote) / 30)

deacceleration_coyote = 0 / ((end_first_acceleration_blower - end_first_acceleration_coyote) / 30) # deacceleration when coyote is catching up

In [798]:
catchup_time = (catchup_frame - wall_hit_frame) / 30 # amount of time it takes of wile to catch up with blower
distance_ahead = max_speed_coyote * catchup_time # distance that the blower was ahead just before it hit the wall

blower_ahead_average_speed = distance_ahead / ((wall_hit_frame - bursts[0]) / 30) # the average speed of the time that the blower moves faster than the coyote. calculated by when the blower first starts moving faster (at the first burst) until it hits the wall

burst_speed_increase = blower_ahead_average_speed / len(bursts) # the amount of speed gained by each burst, assuming each burst is the same size

182.61063924899418
182.61892 0.7515181893004116
3.0369 1.4090327102545848


In [799]:
volume_body = math.pi * (radius_body**2) * height_body # volume of blower in m^3
volume_air = math.pi * ((radius_body - 0.04)**2) * (height_body - 0.04) # volume of air in m^3

volume_top = math.pi * (radius_top**2) * height_top # volume of top of blower in m^3

volume_aluminium = volume_body - volume_air + volume_top # volume of aluminium in m^3

density_aluminium = 2700 # density of aluminium in kg/m^3
mass_aluminium = volume_aluminium * density_aluminium # mass of aluminium in kg

density_air = 1.293 # density of air in kg/m^3
mass_air = volume_air * density_air # mass of air in kg

In [800]:
weight = mass_aluminium + mass_air # weight of blower in kg
normal_force = 9.81 * weight

roll_friction_coefficient = 0.01 # friction coefficient of the ground with the wheels

ground_friction_newton = roll_friction_coefficient * normal_force

frontal_area = 2 * math.pi * radius_body * height_body / 2 # frontal area of blower in m^2

max_air_friction_newton = 0.5 * density_air * (max_speed_coyote ** 2) * 0.47 * frontal_area # air friction in newton at max speed
total_power_newton = ground_friction_newton + max_air_friction_newton # total friction in newton at max speed

In [801]:
dt = 1/30

acceleration_coyote = 0
velocity_coyote = 0
distance_coyote = 0

acceleration_blower = 0
velocity_blower = 0
distance_blower = 0

frames = list(range(first_frame_blower, catchup_frame + 1))
frames_seconds = [(x - first_frame_blower) * dt for x in frames]

acceleration_data_coyote = []
velocity_data_coyote = []
distance_data_coyote = []

acceleration_data_blower = []
velocity_data_blower = []
distance_data_blower = []

In [802]:
def get_acceleration_coyote(current_frame: int) -> float:
    d_accel = 0
    
    if first_frame_coyote <= current_frame < end_first_acceleration_coyote:
        d_accel += first_acceleration_coyote
        
    elif end_first_acceleration_coyote <= current_frame <= end_first_acceleration_blower:
        d_accel += deacceleration_coyote
        
    if current_frame == catchup_frame:
        d_accel = -velocity / dt
        
    return d_accel


def get_acceleration_blower(current_frame: int) -> float:
    d_accel = 0
    
    if first_frame_blower <= current_frame < end_first_acceleration_blower:
        d_accel += first_acceleration_blower

    if current_frame in bursts:
        d_accel +=  burst_speed_increase
        
    if current_frame == wall_hit_frame:
        d_accel = -velocity / dt
        
        
    return d_accel

In [803]:
for i in frames:
    acceleration_coyote = get_acceleration_coyote(i)
    acceleration_blower = get_acceleration_blower(i)

    velocity_coyote += acceleration_coyote * dt
    velocity_blower += acceleration_blower * dt
    
    distance_coyote += velocity_coyote * dt
    distance_blower += velocity_blower * dt
    
    acceleration_data_coyote.append(acceleration_coyote)
    acceleration_data_blower.append(acceleration_blower)
    
    velocity_data_coyote.append(velocity_coyote)
    velocity_data_blower.append(velocity_blower)
    
    distance_data_coyote.append(distance_coyote)
    distance_data_blower.append(distance_blower)

In [804]:
from plotly.offline import init_notebook_mode
import plotly.graph_objects as go

init_notebook_mode(connected=True)

acceleration_graph_coyote = go.Scatter(
    x=frames_seconds,
    y=acceleration_data_coyote,
    name="Versnelling coyote"
)

velocity_graph_coyote = go.Scatter(
    x=frames_seconds,
    y=velocity_data_coyote,
    name="Snelhijd coyote"
)

distance_graph_coyote = go.Scatter(
    x=frames_seconds,
    y=distance_data_coyote,
    name="Afstand coyote"
)

acceleration_graph_blower = go.Scatter(
    x=frames_seconds,
    y=acceleration_data_blower,
    name="Versnelling stofblazer"
)

velocity_graph_blower = go.Scatter(
    x=frames_seconds,
    y=velocity_data_blower,
    name="Snelhijd stofblazer"
)

distance_graph_blower = go.Scatter(
    x=frames_seconds,
    y=distance_data_blower,
    name="Afstand stofblazer"
)

delta_distance_graph = go.Scatter(
    x=frames_seconds,
    y=[distance_data_blower[i] - distance_data_coyote[i] for i in range(len(frames))],
    name="Afstand tussen coyote en stofblazer"
)

acceleration_fig = go.Figure((acceleration_graph_coyote, acceleration_graph_blower), layout=dict(
    title = "Versnelling van de stofblazer",
    xaxis_title="Tijd (s)",
    yaxis_title="Versnelling (m/s^2)",
    autosize=False
))

velocity_fig = go.Figure((velocity_graph_coyote, velocity_graph_blower), layout=dict(
    title = "Snelheid van de stofblazer",
    xaxis_title="Tijd (s)",
    yaxis_title="Snelheid (m/s)",
    autosize=False
))

distance_fig = go.Figure((distance_graph_coyote, distance_graph_blower), layout=dict(
    title = "Afstand van de stofblazer",
    xaxis_title="Tijd (s)",
    yaxis_title="Afstand (m)",
    autosize=False
))

delta_distance_fig = go.Figure(delta_distance_graph, layout=dict(
    title = "Afstand tussen coyote en stofblazer",
    xaxis_title="Tijd (s)",
    yaxis_title="Afstand (m)",
    autosize=False
))

acceleration_fig.show()
velocity_fig.show()
distance_fig.show()
delta_distance_fig.show()

In [805]:
total_frames = catchup_frame - first_frame_blower # total frames. framerate is 30fps
delta_t = 1/30

acceleration = 0
velocity = 0
distance = 0

time_data = []
acceleration_data = []
velocity_data = []
distance_data = []

def get_friction(current_speed: float) -> float:
    air_friction_newton = 0.5 * density_air * (current_speed ** 2) * 0.47 * frontal_area # air friction in newton
    total_friction_newton = ground_friction_newton + air_friction_newton # total friction in newton
    
    return (total_power_newton - total_friction_newton) / weight # netto F gedeeld door m is versnelling

def get_acceleration(current_frame: int) -> float:
    delta_acceleration = 0
    if first_frame_blower <= current_frame <= end_first_acceleration_blower:
        delta_acceleration += first_acceleration_blower

    if current_frame in bursts:
        delta_acceleration += burst_speed_increase

    return delta_acceleration


for i in range(total_frames):
    frame = i + first_frame_blower
    
    acceleration = get_acceleration(frame) - get_friction(velocity)
    velocity += acceleration * delta_t
    distance += velocity * delta_t
    
    acceleration_data.append(acceleration)
    velocity_data.append(velocity)
    distance_data.append(distance)
    time_data.append(frame * delta_t)

acceleration_graph = go.Scatter(
    x=time_data,
    y=acceleration_data,
    name="Versnelling stofzuiger"
)

velocity_graph = go.Scatter(
    x=time_data,
    y=velocity_data,
    name="Snelhijd stofzuiger"
)    
    
distance_graph = go.Scatter(
    x=time_data,
    y=distance_data,
    name="Afstand stofzuiger"
)

acceleration_fig = go.Figure(acceleration_graph, layout=dict(
    title = "Versnelling van de stofzuiger",
    xaxis_title="Tijd (s)",
    yaxis_title="Versnelling (m/s^2)",
    autosize=False
))

velocity_fig = go.Figure(velocity_graph, layout=dict(
    title = "Snelheid van de stofzuiger",
    xaxis_title="Tijd (s)",
    yaxis_title="Snelheid (m/s)",
    autosize=False
))

distance_fig = go.Figure(distance_graph, layout=dict(
    title = "Afstand van de stofzuiger",
    xaxis_title="Tijd (s)",
    yaxis_title="Afstand (m)",
    autosize=False
))

acceleration_fig.show()
velocity_fig.show()
distance_fig.show()